# IMPORTING LIBRARIES



In [ ]:
import pandas as pd
import re
import nltk

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# DATASETS

In [ ]:
c1 = pd.read_csv("corona (1).csv")
c2 = pd.read_csv("corona.csv")
c3 = pd.read_csv("corona_virus.csv")
c4 = pd.read_csv("covid-19.csv")

h1 = pd.read_csv("USAID.csv")
h2 = pd.read_csv("WHO.csv")
h3 = pd.read_csv("World_Health.csv")

m1 = pd.read_csv("covid-19 cure.csv")
m2 = pd.read_csv("covid-19 vaccine.csv")
m3 = pd.read_csv("cure.csv")
m4 = pd.read_csv("fake news.csv")
m5 = pd.read_csv("vaccine.csv")

r1 = pd.read_csv("religion.csv")

In [ ]:
#c1.shape,c2.shape,c3.shape,c4.shape, h1.shape,h2.shape,h3.shape, m1.shape,m2.shape,m3.shape,m4.shape,m5.shape, r1.shape

In [ ]:
c2 = c2.drop(["extended_entities","withheld_in_countries"],axis=1)
h2 = h2.drop(["extended_entities","withheld_in_countries"],axis=1)
m2 = m2.drop(["withheld_in_countries"],axis=1)
m3 = m3.drop(["extended_entities","withheld_in_countries"],axis=1)

In [ ]:
data = pd.concat([c1,c2,c3,c4,h1,h2,h3,m1,m2,m3,m4,m5,r1],axis=0)

In [ ]:
data.shape

(35000, 30)

# SELECTING ROWS

In [ ]:
data.isna().sum()

contributors                 35000
coordinates                  34987
created_at                       0
display_text_range               0
entities                         0
favorite_count                   0
favorited                        0
full_text                        0
geo                          34987
id                               0
id_str                           0
in_reply_to_screen_name      27565
in_reply_to_status_id        27855
in_reply_to_status_id_str    27855
in_reply_to_user_id          27565
in_reply_to_user_id_str      27565
is_quote_status                  0
lang                             0
metadata                         0
place                        34639
possibly_sensitive           30080
quoted_status                34056
quoted_status_id             31579
quoted_status_id_str         31579
retweet_count                    0
retweeted                        0
retweeted_status             14120
source                           1
truncated           

In [ ]:
df = data[["created_at","full_text","favorite_count","id","is_quote_status","retweet_count","source","user"]]

# CLEANING

In [ ]:
df = df.dropna(axis=0)

In [ ]:
#df.isna().sum()

In [ ]:
df.columns

Index(['created_at', 'full_text', 'favorite_count', 'id', 'is_quote_status',
       'retweet_count', 'source', 'user'],
      dtype='object')

In [ ]:
df.head()

,created_at,full_text,favorite_count,id,is_quote_status,retweet_count,source,user
0,2020-06-22 10:14:11,RT @IYCOdisha: Students asked to give exams in...,0,1275009177480388612,False,90,"<a href=""http://twitter.com/download/android"" ...","{'id': 1206265793136091137, 'id_str': '1206265..."
1,2020-06-22 10:14:10,RT @NSUIAssam: Every day Corona cases are incr...,0,1275009174770864130,False,21,"<a href=""https://mobile.twitter.com"" rel=""nofo...","{'id': 1037590693357613056, 'id_str': '1037590..."
2,2020-06-22 10:14:09,RT @AdityaRajKaul: China gave the world Corona...,0,1275009172887592960,True,1496,"<a href=""http://twitter.com/download/iphone"" r...","{'id': 279855461, 'id_str': '279855461', 'name..."
3,2020-06-22 10:14:09,@Nigella_Lawson This is not going to help my c...,0,1275009170798862336,False,0,"<a href=""http://twitter.com/download/iphone"" r...","{'id': 19272739, 'id_str': '19272739', 'name':..."
4,2020-06-22 10:14:09,Even MLA's and officials are affected by Coron...,0,1275009170287104000,False,0,"<a href=""http://twitter.com/download/android"" ...","{'id': 1550694234, 'id_str': '1550694234', 'na..."


In [ ]:
df.to_csv("dropped.csv")

In [ ]:
#extracting hashtags, handles, retweets
hashtags = [re.findall("#\w+_?\w+?",r) for r in df.full_text]
handles = [re.findall("@\w+_?\w+?",r) for r in df.full_text]
retweets = [re.findall("RT (@\w+_?\w+?:)",r) for r in df.full_text]

In [ ]:
#converting 'created_at' to datetime
df.created_at = pd.to_datetime(df.created_at)

In [ ]:
#removing URLs
clean = [re.sub("https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"," ", r) for r in df.full_text]

#remove RT
clean = [re.sub("RT (@\w+:)","",r) for r in clean]

#removing symbols
clean = [re.sub("[\d@#]","", r) for r in clean]

#removing symbols
clean = [re.sub("[^a-zA-Z]"," ", r) for r in clean]

In [ ]:
clean[0:20]

[' Students asked to give exams in the tough times of corona pandemic  The examinations like NEET  JEE  CA  DBT  Net are MCQs M ',
 ' Every day Corona cases are increasing  but still no one is concerned about the students  For the government conducting exami ',
 ' China gave the world Corona  India gave the world Yoga  Corona killed lakhs across the world  Yoga gave strength  power ',
 'Nigella Lawson This is not going to help my corona curves or lockdown legs but looks totally worth it  ',
 'Even MLA s and officials are affected by Corona  think about the students who will go to write their exams    We won t also not get multi specialty hospitals  NoExamsInCovid',
 ' Physically corona kisi ko hua ho ya nahi  Financially corona has hit us all  Mentally as well ',
 ' Every day Corona cases are increasing  but still no one is concerned about the students  For the government conducting exami ',
 ' Students take years to prepare for exams like NEET and JEE  Wouldn t it be unfair if they ar

In [ ]:
#cleaning text
df.full_text = clean

In [ ]:
med = []
for r in df.source:
  med.append(r.split("\"")[4])

In [ ]:
#removing other symbols
med = [re.sub("[></](Twitter)","",r) for r in med]
med = [re.sub("( for )","",r) for r in med]
med = [re.sub("(</a>)","",r) for r in med]
med = [re.sub("\s","",r) for r in med]

In [ ]:
df['Medium'] = med

In [ ]:
# #source column
# device = re.findall("Twitter for (\w+)",df.source.split())
# web = [re.findall("Twitter (Web App)",r) for r in df.source]

# for i in range(len(device)):
#   if(device[i]==[]):
#     device[i] = web[i]
# df['Medium'] = device

In [ ]:
#Name and Location
loc = []
name = []
for r in df.user:
  loc.append(r.split("'")[17])
  name.append(r.split("'")[9])

In [ ]:
#Names of user
df['Name'] = name

In [ ]:
df_new = df.drop(["favorite_count","is_quote_status","source","user"],axis=1)

# FINAL DATASET

In [ ]:
df_new.head()

,created_at,full_text,id,retweet_count,Medium,Name
0,2020-06-22 10:14:11,Students asked to give exams in the tough tim...,1275009177480388612,90,Android,Md Abu Zaid
1,2020-06-22 10:14:10,Every day Corona cases are increasing but st...,1275009174770864130,21,WebApp,🎶Ujjval
2,2020-06-22 10:14:09,China gave the world Corona India gave the w...,1275009172887592960,1496,iPhone,Shrikant Sahu
3,2020-06-22 10:14:09,Nigella Lawson This is not going to help my co...,1275009170798862336,0,iPhone,Brooke Bannister
4,2020-06-22 10:14:09,Even MLA s and officials are affected by Coron...,1275009170287104000,0,Android,Karan.


In [ ]:
#top 4 Medium

In [ ]:
df_new.columns

Index(['created_at', 'full_text', 'id', 'retweet_count', 'Medium', 'Name'], dtype='object')

In [ ]:
loc1 = []
for r in loc:
  if(r!=''):
    loc1.append(r)

In [ ]:
# loc1

In [ ]:
location = pd.DataFrame(loc1,columns=["Location"])

In [ ]:
location.to_csv("location.csv")

In [ ]:
df_new.to_csv("Tweets_data1.csv")